# 모델 선택 및 개선

In [56]:
import os, warnings
# 경고 메세지 무시하거나 숨길때(ignore), 다시보이게(default)
# warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='ignore')

In [57]:
import pandas as pd

## 랜덤 포레스트 모델 선택

In [58]:
from sklearn.ensemble import RandomForestClassifier

### 교차 검증(KFold)

In [59]:
from sklearn.model_selection import cross_val_score, KFold

### 데이터 확인 및 전처리

In [60]:
# 데이터 읽기
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# 결측값 채우기(Age)
train.Age = train.Age.fillna(train.Age.median())
test.Age = test.Age.fillna(train.Age.median())
# 결측값 채우고 숫자형으로 변환(Cabin)
train['Cabin_n'] = train['Cabin'].notnull().astype(int)
test['Cabin_n'] = test['Cabin'].notnull().astype(int)
# 결측값 채우기(train.Embarked)
train.Embarked = train.Embarked.fillna('S')
# 결측값 채우기(test.Fare)
test['Fare'] = test['Fare'].median()

# 데이터 수치형으로 변환(Sex)
train['Sex_n'] = train.Sex.map({'male':1, 'female':2})
test['Sex_n'] = test.Sex.map({'male':1, 'female':2})
# 데이터 수치형으로 변환(Embarked)
train['Embarked_n'] = train['Embarked'].map({'S':1, 'C':2, 'Q':3})
test['Embarked_n'] = test['Embarked'].map({'S':1, 'C':2, 'Q':3})

# 데이터 특성 선택 및 분할
sel = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin_n', 'Sex_n', 'Embarked_n']
X = train[sel]
y = train['Survived']

### 모델 객체화

In [65]:
# 모델 생성
rf_c = RandomForestClassifier(n_jobs=-1)

## 교차 검증 : LeaveOneOut

In [66]:
# test_size 변경
for i in range(1, 6):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify =y, test_size = (i/10), random_state=0)
    # 모델 훈련
    rf_c.fit(X_train,  y_train)
    # 교차검증
    kfold = KFold(shuffle=True)
    # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
    score = cross_val_score(rf_c, X_test, y_test, cv=kfold)
    print(f"훈련 : 테스트 = {10-i} : {i}")
    print(f"교차 검증 점수 : {score}")
    print("교차 검증 점수 평균 :", score.mean())
    print()

훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.66666667 0.61111111 0.88888889 0.72222222 0.72222222]
교차 검증 점수 평균 : 0.7222222222222222

훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.83333333 0.66666667 0.80555556 0.80555556 0.77142857]
교차 검증 점수 평균 : 0.7765079365079364

훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.77777778 0.88888889 0.7037037  0.79245283 0.8490566 ]
교차 검증 점수 평균 : 0.802375960866527

훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.73611111 0.88888889 0.71830986 0.83098592 0.71830986]
교차 검증 점수 평균 : 0.7785211267605633

훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.77777778 0.79775281 0.76404494 0.76404494 0.75280899]
교차 검증 점수 평균 : 0.7712858926342072



## RandomForestClassifier 모델1
* 'Age' : median
* test.fare : median
* statify 설정
* 교차검증 : KFold(shuffle=True)
* 점수
    * 5:5 => 0.7983770287141073
    * 9:1 => 0.6888888888888889

### max_depth, test_size 조절

In [68]:
for d in range(1, 5):
    for i in range(1, 6):
        # 모델 생성
        rf_c = RandomForestClassifier(n_jobs=-1, max_depth=d)
        # 데이터 셋 분할
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify =y, test_size = (i/10), random_state=0)
        # 모델 훈련
        rf_c.fit(X_train,  y_train)
        # 교차검증
        kfold = KFold(shuffle=True)
        # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation)
        score = cross_val_score(rf_c, X_test, y_test, cv=kfold)
        print(f"max_depth = {d}")
        print(f"훈련 : 테스트 = {10-i} : {i}")
        print(f"교차 검증 점수 : {score}")
        print("교차 검증 점수 평균 :", score.mean())
        print()
    print("===================================================================")

max_depth = 1
훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.66666667 0.66666667 0.66666667 0.72222222 0.77777778]
교차 검증 점수 평균 : 0.7

max_depth = 1
훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.72222222 0.69444444 0.88888889 0.72222222 0.77142857]
교차 검증 점수 평균 : 0.7598412698412698

max_depth = 1
훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.77777778 0.81481481 0.77777778 0.66037736 0.75471698]
교차 검증 점수 평균 : 0.7570929419986024

max_depth = 1
훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.77777778 0.70833333 0.8028169  0.70422535 0.70422535]
교차 검증 점수 평균 : 0.7394757433489827

max_depth = 1
훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.72222222 0.71910112 0.82022472 0.70786517 0.69662921]
교차 검증 점수 평균 : 0.7332084893882647

max_depth = 2
훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.72222222 0.77777778 0.66666667 0.72222222 0.66666667]
교차 검증 점수 평균 : 0.711111111111111

max_depth = 2
훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.80555556 0.72222222 0.77777778 0.61111111 0.85714286]
교차 검증 점수 평균 : 0.7547619047619047

max_depth = 2
훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.85185185 0.7962963  0.81481481 0.7924528

### 랜덤 포레스트 분류 모델 최적 하이퍼 파라미터
* max_depth = 3, 4
* test_size = 0.2, 0.3